In [ ]:
import re
import requests
import urllib.parse
import pandas as pd
import numpy as np
import json
from datetime import datetime
TIMEOUT = 10


In [11]:
API_KEY = "RGAPI-8ca87305-91de-46ef-8d0a-717e7071aa93"

In [ ]:
import re
import requests
import urllib.parse
TIMEOUT = 10
API_KEY = "RGAPI-8ca87305-91de-46ef-8d0a-717e7071aa93"

In [34]:
# Updated function using Account API v1 (recommended approach)
def get_puuid_by_riot_id(game_name: str, tag_line: str, api_key: str):
    """
    Get PUUID using Account API v1 by Riot ID (gameName + tagLine)
    This is the newer, recommended approach instead of Summoner API v4
    """
    base_url = "https://americas.api.riotgames.com"
    url = f"{base_url}/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    headers = {"X-Riot-Token": api_key}
    
    try:
        response = requests.get(url, headers=headers, timeout=TIMEOUT)
        if response.status_code == 200:
            return response.json()
        else:
            return {"status_code": response.status_code, "text": response.text}
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# Test the new function
print("Testing Account API v1...")
result = get_puuid_by_riot_id("Arjian", "NA1", API_KEY)
print("Result:", result)


Testing Account API v1...
Result: {'puuid': 'idX_GzoSd0zdsjYdp8rDVByZ53WmJN1uRhHiwnCrKzY2Trabiezrp9rTvTxSJQ3vqkXio1MKTrKMrQ', 'gameName': 'Arjian', 'tagLine': 'NA1'}


In [49]:
# Function to analyze events in a specific timeline frame
def analyze_timeline_events(timeline_data, frame_index=30):
    """
    Analyze events in a specific timeline frame and show unique event types
    
    Args:
        timeline_data: Raw timeline data from Riot API
        frame_index: Index of the frame to analyze (default: 30)
    
    Returns:
        dict: Analysis of events in that frame
    """
    
    frames = timeline_data['info']['frames']
    
    # Check if frame index exists
    if frame_index >= len(frames):
        print(f"Frame index {frame_index} is out of range. Available frames: 0 to {len(frames)-1}")
        return None
    
    frame = frames[frame_index]
    events = frame.get('events', [])
    
    print(f"=== FRAME {frame_index} EVENT ANALYSIS ===")
    print(f"Timestamp: {frame['timestamp']} ms ({frame['timestamp']/1000/60:.1f} minutes)")
    print(f"Total events in this frame: {len(events)}")
    
    if not events:
        print("No events in this frame.")
        return None
    
    # Get unique event types
    event_types = [event.get('type', 'UNKNOWN') for event in events]
    unique_events = list(set(event_types))
    
    print(f"\nUnique event types in frame {frame_index}:")
    print(f"Count: {len(unique_events)}")
    
    for event_type in sorted(unique_events):
        count = event_types.count(event_type)
        print(f"  • {event_type}: {count} occurrence(s)")
    
    # Detailed breakdown of each event type
    print(f"\n=== DETAILED EVENT BREAKDOWN ===")
    
    event_details = {}
    for event in events:
        event_type = event.get('type', 'UNKNOWN')
        if event_type not in event_details:
            event_details[event_type] = []
        
        # Extract relevant details based on event type
        event_info = {
            'timestamp': event.get('timestamp'),
            'participant_id': event.get('participantId'),
            'killer_id': event.get('killerId'),
            'victim_id': event.get('victimId'),
            'position': event.get('position', {}),
            'bounty': event.get('bounty'),
            'lane_type': event.get('laneType'),
            'building_type': event.get('buildingType'),
            'tower_type': event.get('towerType'),
            'ward_type': event.get('wardType'),
            'level': event.get('level'),
            'skill_slot': event.get('skillSlot'),
            'level_up_type': event.get('levelUpType')
        }
        
        event_details[event_type].append(event_info)
    
    # Print details for each event type
    for event_type, events_list in event_details.items():
        print(f"\n--- {event_type} ({len(events_list)} events) ---")
        
        for i, event_info in enumerate(events_list, 1):
            print(f"  Event {i}:")
            for key, value in event_info.items():
                if value is not None and value != {} and value != 0:
                    print(f"    {key}: {value}")
    
    return {
        'frame_index': frame_index,
        'timestamp': frame['timestamp'],
        'total_events': len(events),
        'unique_event_types': unique_events,
        'event_counts': {event_type: event_types.count(event_type) for event_type in unique_events},
        'event_details': event_details
    }

# Function to check events across all frames
def analyze_all_timeline_events(timeline_data):
    """
    Analyze all events across the entire timeline
    """
    frames = timeline_data['info']['frames']
    
    print("=== COMPLETE TIMELINE EVENT ANALYSIS ===")
    print(f"Total frames: {len(frames)}")
    
    all_events = []
    frame_event_counts = []
    
    for i, frame in enumerate(frames):
        events = frame.get('events', [])
        frame_event_counts.append(len(events))
        all_events.extend(events)
    
    # Overall statistics
    print(f"Total events across all frames: {len(all_events)}")
    print(f"Average events per frame: {len(all_events)/len(frames):.1f}")
    print(f"Max events in a single frame: {max(frame_event_counts)}")
    print(f"Min events in a single frame: {min(frame_event_counts)}")
    
    # Unique event types across entire timeline
    all_event_types = [event.get('type', 'UNKNOWN') for event in all_events]
    unique_global_events = list(set(all_event_types))
    
    print(f"\nUnique event types across entire timeline:")
    print(f"Count: {len(unique_global_events)}")
    
    for event_type in sorted(unique_global_events):
        count = all_event_types.count(event_type)
        percentage = (count / len(all_events)) * 100
        print(f"  • {event_type}: {count} events ({percentage:.1f}%)")
    
    return {
        'total_frames': len(frames),
        'total_events': len(all_events),
        'unique_event_types': unique_global_events,
        'event_counts': {event_type: all_event_types.count(event_type) for event_type in unique_global_events},
        'frame_event_counts': frame_event_counts
    }

# Example usage:
# If you have timeline data, uncomment and run:
# frame_analysis = analyze_timeline_events(timeline, 30)
# global_analysis = analyze_all_timeline_events(timeline)


In [20]:
import requests
import pandas as pd

ROUTING = "americas"  # americas | asia | europe | sea  (pick based on the player’s platform region)
BASE = f"https://{ROUTING}.api.riotgames.com"

HEADERS = {"X-Riot-Token": API_KEY}

def get_match_ids(puuid: str, start: int = 0, count: int = 5):
    url = f"{BASE}/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {"start": start, "count": count}
    r = requests.get(url, headers=HEADERS, params=params, timeout=15)
    r.raise_for_status()
    return r.json()  # list[str] of matchIds

def get_timeline(match_id: str):
    url = f"{BASE}/lol/match/v5/matches/{match_id}/timeline"
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    return r.json()  # TimelineDto

def timeline_to_timeseries(timeline: dict):
    """
    Returns two DataFrames indexed by game time (ms):
      - gold_df: columns are participantId (1..10), values = totalGold
      - xp_df:   columns are participantId (1..10), values = xp
    """
    frames = timeline["info"]["frames"]
    rows_gold = []
    rows_xp = []
    for fr in frames:
        t = fr["timestamp"]  # ms
        pf = fr["participantFrames"]  # keys '1'..'10'
        gold_row = {"timestamp_ms": t}
        xp_row = {"timestamp_ms": t}
        for pid_str, pframe in pf.items():
            pid = int(pid_str)
            gold_row[pid] = pframe.get("totalGold", None)
            xp_row[pid] = pframe.get("xp", None)
        rows_gold.append(gold_row)
        rows_xp.append(xp_row)

    gold_df = pd.DataFrame(rows_gold).set_index("timestamp_ms").sort_index()
    xp_df = pd.DataFrame(rows_xp).set_index("timestamp_ms").sort_index()
    return gold_df, xp_df

# ---------- Example usage ----------
# 1) You must already know the player's PUUID (from Summoner-v4 + Account-v1).
#    Once you have PUUID, you can fetch recent match ids:
# puuid = "idX_GzoSd0zdsjYdp8rDVByZ53WmJN1uRhHiwnCrKzY2Trabiezrp9rTvTxSJQ3vqkXio1MKTrKMrQ"
# match_ids = get_match_ids(puuid, count=1)
# print(match_ids)
# match_id = match_ids[0]

# 2) Or if you already have a matchId:
match_id = "NA1_5371187000"

timeline = get_timeline(match_id)
print(timeline)
# gold_df, xp_df = timeline_to_timeseries(timeline)

# print(gold_df.head())
# print(xp_df.head())

# To compute team gold difference over time (Blue=participants 1-5, Red=6-10):
# gold_df["blue_gold"] = gold_df[[1,2,3,4,5]].sum(axis=1)
# gold_df["red_gold"]  = gold_df[[6,7,8,9,10]].sum(axis=1)
# gold_df["gold_diff_blue_minus_red"] = gold_df["blue_gold"] - gold_df["red_gold"]


{'metadata': {'dataVersion': '2', 'matchId': 'NA1_5371187000', 'participants': ['LjSc9W9-X3NjQwgbnKigRS8CKNRc2u9aQjgPt1IndZ5qZ39Na9TMzpZfp8YOYrSpPhuvWWE57pZ6yw', 'idX_GzoSd0zdsjYdp8rDVByZ53WmJN1uRhHiwnCrKzY2Trabiezrp9rTvTxSJQ3vqkXio1MKTrKMrQ', '2j4bF233A34NebhKBs8mJYqv_JguzIhCk-Zy-VH5HBOfLhrxOqYfOZTlvv3cLRPWP09YhOyzqxiYFA', 'r7VWqyRynVrFoIFnwX7ZLqylD29OOLhAVYDgfzonROQgGfFa6O7rnKwnh6KF9iDSvhQ1WsrbivfifQ', 'jHoqVxdxiSTnQaYzWntThi6qkEKPiC-HauRUUCioPSeSu3qvFndG_KwhvAft7G_Skjyb6RICZouOBA', 'L_U1Sg7nGQVlnb9PySx6SJH_Hf7i0qq8zkOTlvtPAmUAStRBRtk2kP7Ni3SGg8itt5rroCgZjT5uiQ', 'dTltUR43473Idpi0qwYVr-iOhpLbCSbxdXKM5bYYEQgeg4w2qrSkNzEy7PVcnIKTpUGkRVluWf4_SA', 'NIAE7nncvE4tllCw-UOeCu9gQyz00Wi8KsQBhFgxBqAC9EzJ_v8VYrVoPpYPRzJkOnt0WZgodmh87g', 'gcP6F_sV4aMbaigOr82TfB2uWXL1rfjcHzwsWzHLIumhoYKT0IjxAfrwHtIKEG6q-88QabGKXo_QKg', '0NHcZzl8g8HLyLfUTkATxzove_UBVfIsZn3gjCedvIXl4QexL1AsUMrvdQxDOCB7m2hKUxV76l-HcA']}, 'info': {'endOfGameResult': 'GameComplete', 'frameInterval': 60000, 'frames': [{'events': [{'realT

In [47]:
timeline['metadata']

{'dataVersion': '2',
 'matchId': 'NA1_5371187000',
 'participants': ['LjSc9W9-X3NjQwgbnKigRS8CKNRc2u9aQjgPt1IndZ5qZ39Na9TMzpZfp8YOYrSpPhuvWWE57pZ6yw',
  'idX_GzoSd0zdsjYdp8rDVByZ53WmJN1uRhHiwnCrKzY2Trabiezrp9rTvTxSJQ3vqkXio1MKTrKMrQ',
  '2j4bF233A34NebhKBs8mJYqv_JguzIhCk-Zy-VH5HBOfLhrxOqYfOZTlvv3cLRPWP09YhOyzqxiYFA',
  'r7VWqyRynVrFoIFnwX7ZLqylD29OOLhAVYDgfzonROQgGfFa6O7rnKwnh6KF9iDSvhQ1WsrbivfifQ',
  'jHoqVxdxiSTnQaYzWntThi6qkEKPiC-HauRUUCioPSeSu3qvFndG_KwhvAft7G_Skjyb6RICZouOBA',
  'L_U1Sg7nGQVlnb9PySx6SJH_Hf7i0qq8zkOTlvtPAmUAStRBRtk2kP7Ni3SGg8itt5rroCgZjT5uiQ',
  'dTltUR43473Idpi0qwYVr-iOhpLbCSbxdXKM5bYYEQgeg4w2qrSkNzEy7PVcnIKTpUGkRVluWf4_SA',
  'NIAE7nncvE4tllCw-UOeCu9gQyz00Wi8KsQBhFgxBqAC9EzJ_v8VYrVoPpYPRzJkOnt0WZgodmh87g',
  'gcP6F_sV4aMbaigOr82TfB2uWXL1rfjcHzwsWzHLIumhoYKT0IjxAfrwHtIKEG6q-88QabGKXo_QKg',
  '0NHcZzl8g8HLyLfUTkATxzove_UBVfIsZn3gjCedvIXl4QexL1AsUMrvdQxDOCB7m2hKUxV76l-HcA']}

In [ ]:
timeline['info']['frames'][0].keys()

dict_keys(['events', 'participantFrames', 'timestamp'])

In [53]:
timeline['info']['frames'][30]['events']

[{'creatorId': 10,
  'timestamp': 1745521,
  'type': 'WARD_PLACED',
  'wardType': 'SIGHT_WARD'},
 {'creatorId': 5,
  'timestamp': 1747826,
  'type': 'WARD_PLACED',
  'wardType': 'SIGHT_WARD'},
 {'killerId': 5,
  'timestamp': 1748126,
  'type': 'WARD_KILL',
  'wardType': 'YELLOW_TRINKET'},
 {'creatorId': 2,
  'timestamp': 1748627,
  'type': 'WARD_PLACED',
  'wardType': 'YELLOW_TRINKET'},
 {'creatorId': 10,
  'timestamp': 1755456,
  'type': 'WARD_PLACED',
  'wardType': 'SIGHT_WARD'},
 {'itemId': 1018,
  'participantId': 6,
  'timestamp': 1759397,
  'type': 'ITEM_DESTROYED'},
 {'itemId': 6670,
  'participantId': 6,
  'timestamp': 1759397,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1055,
  'participantId': 6,
  'timestamp': 1761134,
  'type': 'ITEM_SOLD'},
 {'itemId': 1037,
  'participantId': 6,
  'timestamp': 1761668,
  'type': 'ITEM_PURCHASED'},
 {'assistingParticipantIds': [10],
  'bounty': 126,
  'killStreakLength': 4,
  'killerId': 9,
  'position': {'x': 8142, 'y': 3522},
  'shutdownBou

In [50]:
analyze_all_timeline_events(timeline)

=== COMPLETE TIMELINE EVENT ANALYSIS ===
Total frames: 43
Total events across all frames: 1274
Average events per frame: 29.6
Max events in a single frame: 57
Min events in a single frame: 6

Unique event types across entire timeline:
Count: 17
  • BUILDING_KILL: 13 events (1.0%)
  • CHAMPION_KILL: 104 events (8.2%)
  • CHAMPION_SPECIAL_KILL: 17 events (1.3%)
  • DRAGON_SOUL_GIVEN: 2 events (0.2%)
  • ELITE_MONSTER_KILL: 13 events (1.0%)
  • FEAT_UPDATE: 10 events (0.8%)
  • GAME_END: 1 events (0.1%)
  • ITEM_DESTROYED: 202 events (15.9%)
  • ITEM_PURCHASED: 262 events (20.6%)
  • ITEM_SOLD: 14 events (1.1%)
  • ITEM_UNDO: 17 events (1.3%)
  • LEVEL_UP: 167 events (13.1%)
  • PAUSE_END: 1 events (0.1%)
  • SKILL_LEVEL_UP: 183 events (14.4%)
  • TURRET_PLATE_DESTROYED: 7 events (0.5%)
  • WARD_KILL: 54 events (4.2%)
  • WARD_PLACED: 207 events (16.2%)


{'total_frames': 43,
 'total_events': 1274,
 'unique_event_types': ['ITEM_UNDO',
  'ITEM_DESTROYED',
  'BUILDING_KILL',
  'ITEM_PURCHASED',
  'LEVEL_UP',
  'PAUSE_END',
  'DRAGON_SOUL_GIVEN',
  'CHAMPION_KILL',
  'WARD_KILL',
  'WARD_PLACED',
  'CHAMPION_SPECIAL_KILL',
  'FEAT_UPDATE',
  'ITEM_SOLD',
  'TURRET_PLATE_DESTROYED',
  'SKILL_LEVEL_UP',
  'GAME_END',
  'ELITE_MONSTER_KILL'],
 'event_counts': {'ITEM_UNDO': 17,
  'ITEM_DESTROYED': 202,
  'BUILDING_KILL': 13,
  'ITEM_PURCHASED': 262,
  'LEVEL_UP': 167,
  'PAUSE_END': 1,
  'DRAGON_SOUL_GIVEN': 2,
  'CHAMPION_KILL': 104,
  'WARD_KILL': 54,
  'WARD_PLACED': 207,
  'CHAMPION_SPECIAL_KILL': 17,
  'FEAT_UPDATE': 10,
  'ITEM_SOLD': 14,
  'TURRET_PLATE_DESTROYED': 7,
  'SKILL_LEVEL_UP': 183,
  'GAME_END': 1,
  'ELITE_MONSTER_KILL': 13},
 'frame_event_counts': [6,
  30,
  10,
  35,
  34,
  37,
  25,
  31,
  32,
  38,
  30,
  35,
  37,
  57,
  30,
  26,
  49,
  29,
  29,
  28,
  37,
  45,
  21,
  17,
  47,
  33,
  20,
  42,
  16,
  25,
 

In [54]:
process_timeline_data(timeline)

{'gold_df':      timestamp  participant_id  current_gold  total_gold  team
 0            0               1           500         500  Blue
 1            0               2           500         500  Blue
 2            0               3           500         500  Blue
 3            0               4           500         500  Blue
 4            0               5           100         500  Blue
 ..         ...             ...           ...         ...   ...
 425    2496822               6          2823       17540   Red
 426    2496822               7          1218       14872   Red
 427    2496822               8          1111       12690   Red
 428    2496822               9          4531       22575   Red
 429    2496822              10           482       14366   Red
 
 [430 rows x 5 columns],
 'xp_df':      timestamp  participant_id     xp  level  team
 0            0               1      0      1  Blue
 1            0               2      0      1  Blue
 2            0              

In [36]:
# Function to process match timeline data
def process_timeline_data(timeline_data):
    """
    Process Riot Games match timeline data and extract key metrics
    
    Args:
        timeline_data: Raw timeline data from Riot API
    
    Returns:
        dict: Processed data with various DataFrames and metrics
    """
    
    frames = timeline_data['info']['frames']
    participants = timeline_data['info']['participants']
    
    # Initialize lists to store data
    gold_data = []
    xp_data = []
    health_data = []
    damage_data = []
    position_data = []
    events_data = []
    
    # Process each frame
    for frame in frames:
        timestamp = frame['timestamp']
        participant_frames = frame['participantFrames']
        
        # Extract participant frame data
        for participant_id, pframe in participant_frames.items():
            pid = int(participant_id)
            
            # Gold data
            gold_data.append({
                'timestamp': timestamp,
                'participant_id': pid,
                'current_gold': pframe.get('currentGold', 0),
                'total_gold': pframe.get('totalGold', 0)
            })
            
            # XP data
            xp_data.append({
                'timestamp': timestamp,
                'participant_id': pid,
                'xp': pframe.get('xp', 0),
                'level': pframe.get('level', 1)
            })
            
            # Health data
            champion_stats = pframe.get('championStats', {})
            health_data.append({
                'timestamp': timestamp,
                'participant_id': pid,
                'current_health': champion_stats.get('health', 0),
                'max_health': champion_stats.get('healthMax', 0),
                'health_percentage': champion_stats.get('health', 0) / max(champion_stats.get('healthMax', 1), 1)
            })
            
            # Damage stats
            damage_stats = pframe.get('damageStats', {})
            damage_data.append({
                'timestamp': timestamp,
                'participant_id': pid,
                'total_damage_done': damage_stats.get('totalDamageDone', 0),
                'total_damage_to_champions': damage_stats.get('totalDamageDoneToChampions', 0),
                'total_damage_taken': damage_stats.get('totalDamageTaken', 0),
                'magic_damage_done': damage_stats.get('magicDamageDone', 0),
                'physical_damage_done': damage_stats.get('physicalDamageDone', 0),
                'true_damage_done': damage_stats.get('trueDamageDone', 0)
            })
            
            # Position data
            position = pframe.get('position', {})
            position_data.append({
                'timestamp': timestamp,
                'participant_id': pid,
                'x': position.get('x', 0),
                'y': position.get('y', 0)
            })
        
        # Process events in this frame
        for event in frame.get('events', []):
            events_data.append({
                'timestamp': event.get('timestamp', timestamp),
                'type': event.get('type', 'UNKNOWN'),
                'participant_id': event.get('participantId'),
                'killer_id': event.get('killerId'),
                'victim_id': event.get('victimId'),
                'assisting_participants': event.get('assistingParticipantIds', []),
                'position': event.get('position', {}),
                'bounty': event.get('bounty', 0),
                'lane_type': event.get('laneType'),
                'building_type': event.get('buildingType'),
                'tower_type': event.get('towerType')
            })
    
    # Convert to DataFrames
    gold_df = pd.DataFrame(gold_data)
    xp_df = pd.DataFrame(xp_data)
    health_df = pd.DataFrame(health_data)
    damage_df = pd.DataFrame(damage_data)
    position_df = pd.DataFrame(position_data)
    events_df = pd.DataFrame(events_data)
    
    # Create participant info DataFrame
    participant_info = pd.DataFrame([
        {'participant_id': p['participantId'], 'puuid': p['puuid']} 
        for p in participants
    ])
    
    # Calculate team-based metrics
    gold_df['team'] = gold_df['participant_id'].apply(lambda x: 'Blue' if x <= 5 else 'Red')
    xp_df['team'] = xp_df['participant_id'].apply(lambda x: 'Blue' if x <= 5 else 'Red')
    
    # Calculate team totals
    team_gold = gold_df.groupby(['timestamp', 'team'])['total_gold'].sum().reset_index()
    team_xp = xp_df.groupby(['timestamp', 'team'])['xp'].sum().reset_index()
    
    # Pivot for easier analysis
    team_gold_pivot = team_gold.pivot(index='timestamp', columns='team', values='total_gold').fillna(0)
    team_xp_pivot = team_xp.pivot(index='timestamp', columns='team', values='xp').fillna(0)
    
    # Calculate gold/XP differences
    team_gold_pivot['gold_diff'] = team_gold_pivot['Blue'] - team_gold_pivot['Red']
    team_xp_pivot['xp_diff'] = team_xp_pivot['Blue'] - team_xp_pivot['Red']
    
    return {
        'gold_df': gold_df,
        'xp_df': xp_df,
        'health_df': health_df,
        'damage_df': damage_df,
        'position_df': position_df,
        'events_df': events_df,
        'participant_info': participant_info,
        'team_gold': team_gold_pivot,
        'team_xp': team_xp_pivot,
        'frames_count': len(frames),
        'match_duration_ms': frames[-1]['timestamp'] if frames else 0
    }

# Function to analyze match performance
def analyze_match_performance(processed_data):
    """
    Analyze match performance from processed timeline data
    
    Args:
        processed_data: Output from process_timeline_data()
    
    Returns:
        dict: Analysis results
    """
    
    # Get final stats for each participant
    final_gold = processed_data['gold_df'].groupby('participant_id')['total_gold'].max()
    final_xp = processed_data['xp_df'].groupby('participant_id')['xp'].max()
    final_level = processed_data['xp_df'].groupby('participant_id')['level'].max()
    final_damage = processed_data['damage_df'].groupby('participant_id')['total_damage_done'].max()
    final_damage_to_champions = processed_data['damage_df'].groupby('participant_id')['total_damage_to_champions'].max()
    
    # Create participant summary
    participant_summary = pd.DataFrame({
        'participant_id': final_gold.index,
        'final_gold': final_gold.values,
        'final_xp': final_xp.values,
        'final_level': final_level.values,
        'total_damage': final_damage.values,
        'damage_to_champions': final_damage_to_champions.values,
        'team': ['Blue' if x <= 5 else 'Red' for x in final_gold.index]
    })
    
    # Team performance
    blue_team = participant_summary[participant_summary['team'] == 'Blue']
    red_team = participant_summary[participant_summary['team'] == 'Red']
    
    team_stats = {
        'blue_total_gold': blue_team['final_gold'].sum(),
        'red_total_gold': red_team['final_gold'].sum(),
        'blue_total_xp': blue_team['final_xp'].sum(),
        'red_total_xp': red_team['final_xp'].sum(),
        'blue_avg_level': blue_team['final_level'].mean(),
        'red_avg_level': red_team['final_level'].mean(),
        'blue_total_damage': blue_team['total_damage'].sum(),
        'red_total_damage': red_team['total_damage'].sum()
    }
    
    # Key events analysis
    events_df = processed_data['events_df']
    
    kills = events_df[events_df['type'] == 'CHAMPION_KILL']
    deaths_by_team = kills.groupby(kills['victim_id'].apply(lambda x: 'Blue' if x <= 5 else 'Red')).size()
    
    # Building kills
    building_kills = events_df[events_df['type'] == 'BUILDING_KILL']
    towers_destroyed = building_kills[building_kills['building_type'] == 'TOWER_BUILDING']
    
    # Match duration
    match_duration_minutes = processed_data['match_duration_ms'] / (1000 * 60)
    
    return {
        'participant_summary': participant_summary,
        'team_stats': team_stats,
        'match_duration_minutes': match_duration_minutes,
        'total_kills': len(kills),
        'deaths_by_team': deaths_by_team.to_dict(),
        'towers_destroyed': len(towers_destroyed),
        'events_summary': events_df['type'].value_counts().to_dict()
    }


 #Example usage function
def analyze_sample_timeline(timeline_data):
    """
    Complete analysis of timeline data with example output
    """
    print("=== MATCH TIMELINE ANALYSIS ===")
    print(f"Processing {len(timeline_data['info']['frames'])} frames...")
    
    # Process the data
    processed = process_timeline_data(timeline_data)
    
    # Analyze performance
    analysis = analyze_match_performance(processed)
    
    # Print summary
    print(f"\nMatch Duration: {analysis['match_duration_minutes']:.1f} minutes")
    print(f"Total Events: {len(processed['events_df'])}")
    print(f"Total Kills: {analysis['total_kills']}")
    print(f"Towers Destroyed: {analysis['towers_destroyed']}")
    
    print("\n=== TEAM STATISTICS ===")
    team_stats = analysis['team_stats']
    print(f"Blue Team - Gold: {team_stats['blue_total_gold']:,}, XP: {team_stats['blue_total_xp']:,}, Avg Level: {team_stats['blue_avg_level']:.1f}")
    print(f"Red Team  - Gold: {team_stats['red_total_gold']:,}, XP: {team_stats['red_total_xp']:,}, Avg Level: {team_stats['red_avg_level']:.1f}")
    
    print("\n=== PARTICIPANT SUMMARY ===")
    print(analysis['participant_summary'].to_string(index=False))
    
    print("\n=== EVENT TYPES ===")
    for event_type, count in analysis['events_summary'].items():
        print(f"{event_type}: {count}")
    
    return processed, analysis

# Example of how to use with your timeline data:
# processed_data, analysis = analyze_sample_timeline(your_timeline_data)

In [38]:
analyze_sample_timeline(timeline)

=== MATCH TIMELINE ANALYSIS ===
Processing 43 frames...

Match Duration: 41.6 minutes
Total Events: 1274
Total Kills: 104
Towers Destroyed: 11

=== TEAM STATISTICS ===
Blue Team - Gold: 82,338, XP: 100,761, Avg Level: 17.6
Red Team  - Gold: 82,043, XP: 103,067, Avg Level: 17.8

=== PARTICIPANT SUMMARY ===
 participant_id  final_gold  final_xp  final_level  total_damage  damage_to_champions team
              1       15830     23849           18        205014                41806 Blue
              2       13748     19240           18        308513                20358 Blue
              3       17733     20808           18        216352                46949 Blue
              4       20621     21713           18        311010                61249 Blue
              5       14406     15151           16         37243                25105 Blue
              6       17540     22898           18        238567                50878  Red
              7       14872     22011           18      

({'gold_df':      timestamp  participant_id  current_gold  total_gold  team
  0            0               1           500         500  Blue
  1            0               2           500         500  Blue
  2            0               3           500         500  Blue
  3            0               4           500         500  Blue
  4            0               5           100         500  Blue
  ..         ...             ...           ...         ...   ...
  425    2496822               6          2823       17540   Red
  426    2496822               7          1218       14872   Red
  427    2496822               8          1111       12690   Red
  428    2496822               9          4531       22575   Red
  429    2496822              10           482       14366   Red
  
  [430 rows x 5 columns],
  'xp_df':      timestamp  participant_id     xp  level  team
  0            0               1      0      1  Blue
  1            0               2      0      1  Blue
  2         

In [37]:
process_timeline_data(timeline)

{'gold_df':      timestamp  participant_id  current_gold  total_gold  team
 0            0               1           500         500  Blue
 1            0               2           500         500  Blue
 2            0               3           500         500  Blue
 3            0               4           500         500  Blue
 4            0               5           100         500  Blue
 ..         ...             ...           ...         ...   ...
 425    2496822               6          2823       17540   Red
 426    2496822               7          1218       14872   Red
 427    2496822               8          1111       12690   Red
 428    2496822               9          4531       22575   Red
 429    2496822              10           482       14366   Red
 
 [430 rows x 5 columns],
 'xp_df':      timestamp  participant_id     xp  level  team
 0            0               1      0      1  Blue
 1            0               2      0      1  Blue
 2            0              

In [1]:
import pandas as pd
import sys
import os

# Change to the project root directory
os.chdir('..')
print(f"Changed to directory: {os.getcwd()}")

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

# Now import from the correct path
from src.data.dataloader import (
    LoLSequenceDataset, 
    create_dataloaders, 
    create_data_splits,
    load_data_splits,
    collate_fn
)
pd.read_parquet('data/splits/train.parquet')

Changed to directory: /Users/Chenjunyu/Desktop/Work/ESE5380_FINAL


,match_id,frame_idx,timestamp,Total_Gold_Difference,Total_Xp_Difference,Total_Gold_Difference_Last_Time_Frame,Total_Xp_Difference_Last_Time_Frame,Total_Minions_Killed_Difference,Total_Jungle_Minions_Killed_Difference,Magic_Damage_Done_Diff,...,Player10_Power_Percent,Player10_Power_Regen,Player10_Movement_Speed,Player10_X_Position,Player10_Y_Position,CentroidDist,MinInterTeamDist,EngagedDiff,FrontlineOverlap,RadialVelocityDiff
0,NA1_5352371307,0,0,0,0,0.0,0.0,0,0,0,...,1.000000,21,325,14281,14199,19711.311879,19335.988545,0.0,-39047.187864,2.785328e-13
1,NA1_5352371307,1,60001,0,0,0.0,0.0,0,0,0,...,1.000000,21,491,13982,12766,5420.723642,1419.884502,0.0,-6353.057465,3.637979e-13
2,NA1_5352371307,2,120018,-2,35,0.0,0.0,-1,0,-1516,...,1.000000,21,334,13233,2193,829.497800,558.054657,0.0,7516.398960,7.275958e-13
3,NA1_5352371307,3,180108,279,113,-2.0,35.0,6,-4,-2760,...,0.884746,22,334,12722,3079,2064.244094,741.326514,0.0,1031.607030,4.320100e-13
4,NA1_5352371307,4,240137,318,-257,279.0,113.0,2,0,-4632,...,1.000000,24,394,11758,11223,8331.556383,936.436330,0.0,-8096.946628,-6.821210e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90010,NA1_5398549489,34,2040628,80,858,163.0,1057.0,95,-7,176366,...,0.864516,285,380,11723,7754,2344.809843,65.192024,0.0,-820.291592,-6.593837e-13
90011,NA1_5398549489,35,2091281,2938,5051,80.0,858.0,104,-3,194660,...,1.000000,127,380,14340,14391,249.677953,237.918053,3.0,1408.046311,-6.025402e-13
90012,NA1_5398551994,0,0,0,0,0.0,0.0,0,0,0,...,1.000000,28,325,14281,14199,19711.311879,19335.988545,0.0,-39047.187864,2.785328e-13
90013,NA1_5398551994,1,60021,0,0,0.0,0.0,0,0,0,...,1.000000,28,334,13203,4512,4574.190468,1399.657101,0.0,-4361.587690,-9.606538e-13
